### OpenAI Assistant API Code

In [ ]:
# API key 환경 설정

import json

OPEN_API_KEY = "your own key"#"your own key"

# obj 모델을 json 전환 후 출력
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

#### Assistant 생성

In [ ]:
# 기 생성 assistant ID 저장
ASSISTANT_ID = 'asst_dlHb89twkKEPKnZTeCguREAC'

In [ ]:
from openai import OpenAI

# client 객체 생성
client = OpenAI(api_key=OPEN_API_KEY)

# Assistant 생성 (Robot 이름, 역할, 모델 지정)
assistant = client.beta.assistants.create(
    name = "Science Tutor",
    instructions = "You are a personal science tutor. Answer questions briefly, in a sentence.",
    model = "gpt-4-turbo-preview"
)

#생성 assistant(Robot) 정보 출력
show_json(assistant)
ASSISTANT_ID = assistant.id

{'id': 'asst_dlHb89twkKEPKnZTeCguREAC',
 'created_at': 1708578593,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal science tutor. Answer questions briefly, in a sentence.',
 'metadata': {},
 'model': 'gpt-4-turbo-preview',
 'name': 'Science Tutor',
 'object': 'assistant',
 'tools': []}

#### Thread 생성

In [ ]:
# 기 생성 Thread ID 저장
THREAD_ID = "thread_LKwqIeZzSUP2gbGL8Vk5L4hl"

In [ ]:
thread = client.beta.threads.create()

show_json(thread)

THREAD_ID = thread.id

{'id': 'thread_LKwqIeZzSUP2gbGL8Vk5L4hl',
 'created_at': 1708579057,
 'metadata': {},
 'object': 'thread',
 'tool_resources': []}

#### Thread의 메시지 생성 및 실행
1. thread에 message 추가
2. thread 실행(run)
3. thread 상태 확인 (대기중, 작업중, 완료, etc)
4. thread에서 최신 message 조회 후 결과확인

In [ ]:
import time

# thread에 message 추가 및 실행
def submit_message(assistant_id, thread_id, user_message):
    # thread message 추가
    client.beta.threads.messages.create(
        thread_id = thread_id, role = "user", content = user_message
    )
    # thread 실행
    run = client.beta.threads.runs.create(
        thread_id = thread_id,
        assistant_id = assistant_id,
    )
    return run

# thread run 실행 중 최신정보로 업데이트
def update_on_run(run, thread_id):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id = thread_id,
            run_id = run.id,
        )
        time.sleep(0.5)
    return run

# thread에 종속된 message 정보 조회 및 출력
def print_thread_message(thread_id):
    responses = client.beta.threads.messages.list(thread_id=thread_id, order="asc")
    for response in responses:
        print(f"[{response.role.upper()}]\n{response.content[0].text.value}\n")


# thread assistant 에게 message를 추가하고 실행
def run_thread_with_message(assistant_id, thread_id, user_message):
    # submit message
    run = submit_message(assistant_id, thread_id, user_message)
    # update_on_run
    run = update_on_run(run, thread_id)
    # list messages
    print_thread_message(thread_id)

    return run


#### Assistant에게 메시지 생성 및 실행 코드 실행 예시

In [ ]:
assistant_id = ASSISTANT_ID
thread_id = THREAD_ID
run = run_thread_with_message(assistant_id, thread_id, "How long it will take by common adult walking from seoul to daejeon?")

[USER]
How long it will take by common adult walking from seoul to daejeon?

[ASSISTANT]
Walking from Seoul to Daejeon, covering a distance of roughly 160 kilometers, would take approximately 32 hours, assuming an average walking speed of 5 kilometers per hour without breaks.

[USER]
How long it will take by common adult walking from seoul to daejeon?

[ASSISTANT]
Walking from Seoul to Daejeon, covering a distance of roughly 160 kilometers, would typically take about 30-35 hours at a continuous moderate pace without considering breaks or overnight stops.

[USER]
How long it will take by common adult walking from seoul to daejeon?

[ASSISTANT]
Walking from Seoul to Daejeon, covering a distance of roughly 160 kilometers, would realistically take about 32 to 35 hours, assuming a continuous walking pace of 5 kilometers per hour without taking into account the need for rests and overnight stops.

